In [164]:
import numpy as np

In [165]:
class Layer:
    def __init__(self) -> None:
        self.input = None
        self.output = None
        pass

    def forward(self, input):
        pass
    
    def backward(self, output_derivative, lr):
        pass

    def update(self, lr):
        self.weights -= lr * self.grad_weights
        self.biases -= lr * self.grad_biases
        pass

In [166]:
class FullLayer(Layer):
    def __init__(self, input_size, output_size) -> None:
        self.weights = np.random.randn(input_size, output_size)
        self.biases = np.random.randn(output_size)
        self.grad_weights = np.zeros_like(self.weights)
        self.grad_biases = np.zeros_like(self.biases)

    def forward(self, input):
        self.input = input
        print (input.shape, self.weights.shape)
        self.output = np.dot(input, self.weights) + self.biases
        print(self.biases.shape)
        print (self.output.shape)
        return self.output
        
    def backward(self, cost_derivatives):
        weighted_sum_derivatives = self.input
        activation_derivatives = self.sigmoid_derivative(self.output)
        #print (cost_derivatives.shape, self.input.shape)
        self.grad_biases = cost_derivatives * activation_derivatives
        
        #TODO: check this
        self.grad_weigts = np.dot(cost_derivatives, self.input.T)
    
        #TODO: check this
        return cost_derivatives * activation_derivatives * weighted_sum_derivatives

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)

In [167]:
def kernel_weight(kernel_size):
    return np.random.randn(kernel_size)

def kernel_bias(input_size, kernel_size):
    input_size = input_size[:2]
    kernel_size = kernel_size[:2]
    output_size = tuple(map(lambda i, j: i - j + 1, input_size, kernel_size)) # Valid padding
    return np.random.randn(output_size)

In [168]:
class SingleConvLayer(Layer): # Valid padding
    def __init__ (self, kernel_sizes : list, input_size : tuple) -> None:
        self.weigths = [kernel_weight(kernel_size) for kernel_size in kernel_sizes]
        self.biases = [kernel_bias(input_size, kernel_size) for kernel_size in kernel_sizes]
        #self.weighted_sum = None
        self.grad_weights = np.zeros_like(self.weights)
        self.grad_biases = np.zeros_like(self.bias)

    def forward(self, input):
        self.input = input
        self.output = [np.correlate(input, kernel) + bias for kernel, bias in zip(self.kernels_weights, self.kernels_bias)]
        return self.output
    
    #def backward(self, cost_derivatives):
        


In [169]:
class Network:
    def __init__(self,layers : list) -> None: 
        self.layers = layers

    def forward(self, inputs : np.ndarray) -> np.ndarray:
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs
    
    def backward(self, output : np.ndarray, desired_output : np.ndarray) -> None:
        cost_derivatives = 2 * (output - desired_output)
        for layer in reversed(self.layers):
            cost_derivatives = layer.backward(cost_derivatives)
            # Derivatives with respect to trainable parameters are saved in indivisual layers
    
    def update(self, lr : float) -> None:
        for layer in self.layers:
            layer.update(lr)

In [170]:
# Define structure of the network
network = Network(
    [
        FullLayer(10, 7)
    ]
)

output = network.forward(np.random.randn(10))
desired_output = np.random.randn(7)
print(output.shape, desired_output.shape)
network.backward(output, desired_output)

(10,) (10, 7)
(7,)
(7,)
(7,) (7,)


ValueError: shapes (7,) and (10,) not aligned: 7 (dim 0) != 10 (dim 0)